# Build a training dataset for ML-based $\Delta$FAC from multiple SNOWPACK simulations
The training dataset needs columns of the following (where each row represents one example):
1. 91 day $\Delta$FAC (m)
2. 91 day average precipitation (mm/day)
3. 1980 - 2021 precipitation climatology (mm/day)
4. 91 day average temperature (C)
5. 1980 - 2021 temperature climatology (C)
6. (10 day precip before tf) - (10 day precip before t0) (mm)

In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd

# First, get a list of SNOWPACK sites that have completed their simulation

In [2]:
%%bash
cd /scratch/summit/erke2265/mass-balance/100_sites/output/
ls -- *.sno | wc -l
basename -s .sno -- *.sno > /scratch/summit/erke2265/mass-balance/notebooks/finished.txt

97


# Next, load an array of strings composed of the finished sites

In [3]:
sites = np.loadtxt("finished.txt", dtype=str)[0:2]
sites

array(['-66.500_103.750_SPINUP', '-67.500_123.125_SPINUP'], dtype='<U23')

# Now get training data

In [4]:
def read_smet(path, var):

    """ Reads a .smet file and returns a time series of the defined variable as a pandas data frame.
    Args:
        path (str): String pointing to the location of the .smet file to be read.
        var  (str): Variable you want to plot
    Returns:
        Time series of defined variable as a pandas data frame.
    """

    # Load .smet file as a Pandas data frame
    df = pd.read_csv(path)

    # Determine indices for data retrieval
    bump = 2

    fields_row = np.where(df[df.columns[0]].str.startswith("fields"))[0][0] + bump
    data_row = np.where(df[df.columns[0]] == '[DATA]')[0][0] + bump
    fields =  np.loadtxt(path, skiprows=fields_row - 1, max_rows=1, dtype='str')
    data_col = []
    for k in range(0, len(var)):
        for j in range(0, len(list(fields[2:]))):
            if var[k] == fields[2+j]:
                data_col.append(j)

    # Creates pandas data frame
    time = np.loadtxt(path, skiprows=data_row, usecols=0, dtype = 'str')
    time = pd.to_datetime(time, format='%Y-%m-%dT%H:%M:%S')
    data = np.loadtxt(path, skiprows=data_row, usecols=tuple(data_col))
    df = pd.DataFrame(data, index=time)

    # Set no data values to nan
    df[df == -999] = np.nan

    # Return time series as daily average Pandas data frame
    return df

def calc_FAC(HS, SWE):
    
    """
    SNOWPACK assumes the density of ice is 917 kg/m^3
    """
    
    SWE_in_units_ice_equivalent = SWE / 0.917
    FAC = HS - SWE_in_units_ice_equivalent
    return FAC

def calc_training_data(site):
    
    # File paths
    meteo_smet = "/scratch/summit/erke2265/mass-balance/100_sites/smet/" + site[:-7]+ ".smet"
    snowpack_smet = "/scratch/summit/erke2265/mass-balance/100_sites/output/" + site[:]+ ".smet"
    
    # Meteo data
    meteo_df = read_smet(meteo_smet, ["TA", "U", "V", "PSUM"])
    snowpack_df = read_smet(snowpack_smet, ["HS_mod", "SWE"])
    
    # Temperature
    ta = meteo_df[0].resample('D').mean()
    ta_mean = ta.mean() # Use in traning
    ta_91_day_mean = ta.rolling(91).mean() # Use in traning

    # Precip
    psum = meteo_df[3].resample('D').mean()
    psum_mean = psum.mean() # Use in traning
    psum_91_day_mean = psum.rolling(91).mean() # Use in traning
    
    # 10 day precip
    precip_10_day_mean = psum.rolling(10).mean()
    precip_10_day_mean_diff = precip_10_day_mean.diff(periods=91) # Use in traning
    
    # Change in FAC
    HS = snowpack_df[0] / 100 # Units: m
    SWE = snowpack_df[1] / 1000 # Units: m

    HS = HS.resample('D').mean()
    SWE = SWE.resample('D').mean()

    FAC = calc_FAC(HS, SWE)
    delta_FAC_91_day = FAC.diff(periods=91) # Use in traning
    
    # Wind Speed
    u = meteo_df[1].resample('D').mean()
    u_mean = u.mean()
    v = meteo_df[2].resample('D').mean()
    v_mean = v.mean()
    ws_mean = np.sqrt(u_mean**2 + v_mean**2) # Use in traning
    
    # Return data
    return ta_mean, ta_91_day_mean, psum_mean, psum_91_day_mean, precip_10_day_mean_diff, delta_FAC_91_day, ws_mean

def read_smet_lat_lon_ratio(path):

    """ Reads a .smet file and returns a time series of the defined variable as a pandas data frame.
    Args:
        path (str): String pointing to the location of the .smet file to be read.
        var  (str): Variable you want to plot
    Returns:
        Time series of defined variable as a pandas data frame.
    """

    # Load .smet file as a Pandas data frame
    df = pd.read_csv(path)

    lat_row = np.where(df[df.columns[0]].str.startswith("latitude"))[0][0]
    lon_row = np.where(df[df.columns[0]].str.startswith("longitude"))[0][0]
    ratio_row = np.where(df[df.columns[0]].str.startswith("units_multiplier"))[0][0]
    
    lat = float(df['SMET 1.1 ASCII'][lat_row].split()[-1])
    lon = float(df['SMET 1.1 ASCII'][lon_row].split()[-1])
    ratio = float(df['SMET 1.1 ASCII'][ratio_row].split()[-2])
    if np.isnan(ratio):
        ratio = 1

    return lat, lon, ratio

In [ ]:
# Initialize training array
train = np.array([])

# Sampling index (hacky... sorry)
ind_1980 = -365 * 42 + 89 # After using x5

# Loop over each site and append training data
for j in range(0, len(sites)):
    print(j)
    
    # Get data
    ta_mean, ta_91_day_mean, psum_mean, psum_91_day_mean, precip_10_day_mean_diff, delta_FAC_91_day, ws_mean = calc_training_data(sites[j])
    
    # Get sin and cos of day of year (cyclic feature)
    doy = np.zeros(len(ta_91_day_mean.index)); doy[:] = np.nan # day of year
    for k in range(0, len(ta_91_day_mean.index)):
        doy[k] = ta_91_day_mean.index[k].timetuple().tm_yday
    day_sin = np.sin(doy*(2.*np.pi/365))
    day_cos = np.cos(doy*(2.*np.pi/365))
    
    # Get lat, lon, and ratio
    smet_path = "/scratch/summit/erke2265/mass-balance/100_sites/smet/" + sites[j][:-7]+ ".smet"
    lat, lon, ratio = read_smet_lat_lon_ratio(smet_path)
    
    # Target data
    Y = np.squeeze(delta_FAC_91_day[ind_1980:].values)

    # Features
    x1 = np.squeeze(psum_91_day_mean[ind_1980:].values)
    x2 = np.repeat(psum_mean, len(x1))
    x3 = np.squeeze(ta_91_day_mean[ind_1980:].values)
    x4 = np.repeat(ta_mean, len(x1))
    x5 = np.squeeze(precip_10_day_mean_diff[ind_1980:].values)
    x6 = np.repeat(ratio, len(x1))
    x7 = day_sin[ind_1980:]
    x8 = day_cos[ind_1980:]
    x9 = np.repeat(ws_mean, len(x1))
    
    # Combine features into readibly savable format
    rand_ind = np.random.choice(np.arange(0, len(x1)), int(len(x1) / 10), replace=False)
    site_lat_lon = np.array([[lat, lon]])
    site_train = np.transpose(np.stack([Y, x1, x2, x3, x4, x5, x6, x7, x8, x9]))
    site_train = site_train[rand_ind, :]
    
    # Single array
    if j == 0: # First site 
        lat_lon = site_lat_lon
        train = site_train
    else: # All other sites
        lat_lon = np.concatenate([lat_lon, site_lat_lon])
        train = np.concatenate([train, site_train])
        
print(train.shape)

0


# Shuffle and save training array

In [ ]:
np.random.shuffle(train)
print(train.shape)
np.savetxt("train.txt", train)

# Make map

In [ ]:
# Get coastlines
df = gpd.read_file("/pl/active/nasa_smb/Data/ADD_Coastline_low_res_polygon.shp")
crs_epsg = ccrs.SouthPolarStereo()
df_epsg = df.to_crs(epsg='3031')

# Generate figure 
fig, axs = plt.subplots(1, 1, subplot_kw={'projection': crs_epsg},
                        figsize=(15, 15))

# Plot sites
plt.scatter(lat_lon[:,1], lat_lon[:,0], linewidth=3, transform=ccrs.PlateCarree())


# Plot coastlines
axs.set_extent((-180, 180, -90, -65), ccrs.PlateCarree())
axs.add_geometries(df_epsg['geometry'], crs=crs_epsg,
                      facecolor='none', edgecolor='black')
print(lat_lon.shape)